[View in Colaboratory](https://colab.research.google.com/github/andersy005/cuda-programming/blob/master/getting_started_on_colab.ipynb)

In [1]:
!apt update -qq

6 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [2]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

--2018-06-27 18:46:54--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?Oy-tuxd8APgk75C-6ni3GsZYC8MKSx8lCk8BAhMEzptmEKWqiBU80Z9TWX4lJfCr-9n4M6xR8eAQcu5bANJUkw92M88T3sQSG2Q5CzCeAhG3ye37lu2a4s6ej_RdyKJ5nHPAmPPd3wAoF-hVGKyZghC3EKpAvO4xKIEOrqItL1bQbfbUalWFWE6JB5e5i2kZ38Qeu_Hz2HpSo1htmYiBXUHogg [following]
--2018-06-27 18:46:54--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?Oy-tuxd8APgk75C-6ni3GsZYC8MKSx8lCk8BAhMEzptmEKWqiBU80Z9TWX4lJfCr-9n4M6xR8eAQcu5bANJUkw92M88T

cuda-repo-ubuntu160 100%[===================>]   1.78G   155MB/s    in 13s     

2018-06-27 18:47:07 (139 MB/s) - ‘cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb.4’ saved [1913589814/1913589814]



In [0]:
!dpkg -i cuda-repo-ubuntu1604–8–0-local-ga2_8.0.61–1_amd64-deb 2> /dev/null

In [4]:
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub

OK


In [0]:
!apt-get update -qq

In [6]:
!apt --fix-broken install
!apt-get install cuda gcc-5 g++-5 -y -qq;

Reading package lists... Done
Building dependency tree       
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [7]:
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'cuda-8-0' for regex 'cuda-8.0'
Note, selecting 'libcuda-8.0-1' for regex 'cuda-8.0'
Note, selecting 'libcuda1-384' instead of 'libcuda-8.0-1'
libcuda1-384 is already the newest version (384.130-0ubuntu0.17.10.1).
cuda-8-0 is already the newest version (8.0.61-1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [9]:
!apt install gcc-5 g++-5 -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gcc-5 is already the newest version (5.5.0-1ubuntu2).
g++-5 is already the newest version (5.5.0-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [11]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [12]:
%%file version.cu
#include <thrust/version.h>
#include <iostream>

int main(void)
{
  int major = THRUST_MAJOR_VERSION;
  int minor = THRUST_MINOR_VERSION;

  std::cout << "Thrust v" << major << "." << minor << std::endl;

  return 0;
}

Writing version.cu


In [13]:
!nvcc version.cu -o version
!./version

nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
Thrust v1.8


In [14]:
%%file thrust_example.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <algorithm>
#include <cstdlib>

int main(void)
{
  // generate 32M random numbers serially
  thrust::host_vector<int> h_vec(32 << 20);
  std::generate(h_vec.begin(), h_vec.end(), rand);

  // transfer data to the device
  thrust::device_vector<int> d_vec = h_vec;

  // sort data on the device 
  thrust::sort(d_vec.begin(), d_vec.end());

  // transfer data back to host
  thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());

  return 0;
}

Writing thrust_example.cu


In [15]:
!nvcc thrust_example.cu -o thrust_example
!./thrust_example


nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [16]:
!nvprof ./thrust_example

==1986== NVPROF is profiling process 1986, command: ./thrust_example
==1986== Profiling application: ./thrust_example
==1986== Profiling result:
Time(%)      Time     Calls       Avg       Min       Max  Name
 30.98%  26.865ms         1  26.865ms  26.865ms  26.865ms  [CUDA memcpy HtoD]
 22.54%  19.542ms         4  4.8854ms  3.8770ms  5.2403ms  void thrust::system::cuda::detail::cub_::DeviceRadixSortDownsweepKernel<thrust::system::cuda::detail::cub_::DeviceRadixSortDispatch<bool=0, int, thrust::system::cuda::detail::cub_::NullType, int>::PtxDownsweepPolicy, bool=0, int, thrust::system::cuda::detail::cub_::NullType, int>(thrust::system::cuda::detail::cub_::NullType*, thrust::system::cuda::detail::cub_::NullType, int*, int, thrust::system::cuda::detail::cub_::DeviceRadixSortDispatch<bool=0, int, thrust::system::cuda::detail::cub_::NullType, int>::PtxDownsweepPolicy*, int*, int, int, bool, bool, thrust::system::cuda::detail::cub_::GridEvenShare<int*>)
 21.37%  18.534ms         1  18.534ms 